In [12]:
# 第一步 从enron6中提取整合spam和ham的邮件数据
import os
import pandas as pd

filepath = "/Users/qiaoqiaoqiaoqiaoqiao/Desktop/研一上/机器学习与神经网络导论/期末试题/enron6/"
text_df = {"subject": [], "content": [], "spam": []}
for file in os.listdir(filepath + "spam"):
    with open(filepath + "spam/" + file, "r", encoding='UTF-8', errors='ignore') as f:
        lines = f.readlines()
        text_df['subject'].append(lines[0][9:].replace("\n", ""))
        text_df['content'].append("".join(lines[1:]))
        text_df["spam"].append(1)

for file in os.listdir(filepath + "ham"):
    with open(filepath + "ham/" + file, "r", encoding='UTF-8') as f:
        lines = f.readlines()
        text_df['subject'].append(lines[0][9:].replace("\n", ""))
        text_df['content'].append("".join(lines[1:]))
        text_df["spam"].append(0)
text_df = pd.DataFrame(text_df)
print(sum(text_df["spam"])) # spam 4500封， ham1500封
text_df

4500


,subject,content,spam
0,"friend , never be in pain again",is your sister in pain\nv ^ 1 ' c . o . d . 1 ...,1
1,"if you can ' t beat em , join em *",gobbledygook . squamous if gobble psychoacoust...,1
2,* detenga la caida del cabello,tnico capilar\nsolucin a la\ncalvicie\ncon la ...,1
3,"dear sir , i am interested in it",hi : )\ndo you need some softwares ? i can giv...,1
4,check out the photos ! - m 20,you got to see this ! i was trying to pass my ...,1
...,...,...,...
5995,bullets 9 / 1,el paso pipeline update - the us dept . of tra...,0
5996,eog material,i understand that eog transferred the funds ye...,0
5997,associate / analyst fall recruiting,correction to super saturday dates\ntwo dates ...,0
5998,tw bullets 1 / 26,capacity marketing - sempra renegotiated a fir...,0


In [13]:
# 第二步 提取特征 分为训练集、测试集
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

X_train_raw, X_test_raw, Y_train, Y_test = train_test_split(text_df[["subject", 'content']],
                                                            text_df[["spam"]],
                                                            test_size = 0.2,
                                                            random_state = 1000)
fe1 = TfidfVectorizer(max_features = 4000)
fe2 = CountVectorizer(max_features = 4000)
X_train_subject, X_train_content = fe1.fit_transform(X_train_raw["subject"]).toarray(), fe2.fit_transform(
    X_train_raw["content"]).toarray()
X_test_subject, X_test_content = fe1.transform(X_test_raw["subject"]).toarray(), fe2.transform(
    X_test_raw["content"]).toarray()
X_train = np.concatenate((X_train_subject, X_train_content), axis = 1)
X_test = np.concatenate((X_test_subject, X_test_content), axis = 1)

In [14]:
# 随机森林（集成学习）选取最优参数
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

parameters = {"n_estimators": [10, 20, 50, 100]}
clf = RandomForestClassifier()
grid = GridSearchCV(clf, parameters, scoring = "accuracy")
grid.fit(X_train, Y_train)
grid.best_params_

/Users/qiaoqiaoqiaoqiaoqiao/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/qiaoqiaoqiaoqiaoqiao/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/qiaoqiaoqiaoqiaoqiao/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/qiaoqiaoqiaoqiaoqiao/opt/anaconda3/lib/pyt

{'n_estimators': 50}

In [15]:
# 第三步 模型中代入最优参数，得到分类评估结果
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators = 50)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
print(classification_report(Y_pred, Y_test))

/var/folders/t4/ry52w44x5cx1tw8blth644fw0000gn/T/ipykernel_60931/1932289359.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, Y_train)


              precision    recall  f1-score   support

           0       0.94      0.99      0.97       267
           1       1.00      0.98      0.99       933

    accuracy                           0.98      1200
   macro avg       0.97      0.99      0.98      1200
weighted avg       0.99      0.98      0.99      1200



In [16]:
# 可知，该垃圾邮箱识别系统的accuracy准确率高达98%